In [12]:
import __init__

from collections import defaultdict
from functools import partial
from typing import Callable

import numpy as np
from cv2.typing import MatLike
from eval import cv2_to_pil, load_data, pil_to_cv2
from eval_with_ground import evaluate_image_pair_from_pil
from eval_without_ground import evaluate_image
from tqdm import tqdm

from common import DataSets
from dehaze import dehaze_and_enhance
from gan import gan_method

In [2]:
paired_samples = load_data()

100%|██████████| 7200/7200 [01:12<00:00, 99.11it/s] 


In [13]:
def eval(dataset: DataSets, method: Callable[[MatLike], MatLike]):
    samples = paired_samples[dataset]
    results = defaultdict(list)
    for sample in tqdm(samples):
        low_image = sample["low_image"]
        high_image = sample.get("high_image")
        # print(low_image, high_image)
        if low_image is not None:
            reference_img = cv2_to_pil(method(pil_to_cv2(low_image)))
            # reference_img.show()
            if high_image is not None:
                scores = evaluate_image_pair_from_pil(reference_img, high_image)
                # Compute metrics here
            else:
                scores = evaluate_image(reference_img, low_image)
            for key, s in scores.items():
                results[key].append(s)
    return results

In [4]:
paired_samples.keys()

dict_keys(['lol_dataset', 'Dark_Face', 'LOL-v2'])

### dahaze

In [5]:
result = eval('lol_dataset', dehaze_and_enhance)

100%|██████████| 500/500 [00:19<00:00, 26.07it/s]


In [6]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3876.0293 ± 3862.0581
PSNR: 14.2037 ± 4.2630
SSIM: 0.5057 ± 0.1516


In [7]:
result = eval('Dark_Face', dehaze_and_enhance)

100%|██████████| 6000/6000 [05:43<00:00, 17.49it/s]


In [8]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 0.9235 ± 6.6518
CEI: 1.8999 ± 0.4300


In [14]:
result = eval('LOL-v2', dehaze_and_enhance)

100%|██████████| 100/100 [00:03<00:00, 33.15it/s]


In [15]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2107.1069 ± 1780.1074
PSNR: 16.2354 ± 3.3389
SSIM: 0.5460 ± 0.1234


In [11]:
dataset = 'lol_dataset'
result = eval(dataset, partial(gan_method, dataset))

  0%|          | 0/500 [00:00<?, ?it/s]


TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)


In [ ]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")